# Gep top20 recommendation - XAI recsys: 
## Model: EBM_extended
## Ranking data: random ranking data

In [1]:
import caffeine
caffeine.on(display=False)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Run this for reproduce
# Gets the current working directory
import os
cwd = os.getcwd()
print("Working directory:", cwd)
# Go up one directory from working directory
os.chdir("..")

Working directory: /Users/anhtth/Library/CloudStorage/OneDrive-UniversityofTwente/2023 UT- THESIS/1-Code/0.cb12_main/nb_recsys_ebm


In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Load ranking data (generated by KNN-TFIDF features)
ranking_data = pd.read_csv('./nb_recsys_ebm/ranking_data_random.csv')

In [6]:
ranking_data.head()

,UserID,JobID,label,City,State,Country
0,13,821691,1,0.0,1.0,1.0
1,13,701157,0,0.0,0.0,0.0
2,13,472398,0,0.0,0.0,0.0
3,13,411244,0,0.0,0.0,0.0
4,13,868940,0,0.0,0.0,0.0


In [7]:
features_df = pd.read_csv('./nb_recsys_ebm/features_df_lda.csv')

In [8]:
features_df.head()

,UserID,JobID,City,State,Country,DegreeType,CurrentlyEmployed,ManagedOthers,WorkHistoryTopic,WorkHistoryLevel,SeniorLevel,ReqTopic,DescTopic,TitTopic
0,7,309823,1.0,1.0,1.0,1,1,0,0,1,4.0,18,2,0
1,7,703889,1.0,1.0,1.0,1,1,0,0,1,4.0,18,2,0
2,7,566574,0.0,0.0,1.0,1,1,0,0,1,4.0,0,18,0
3,7,481216,0.0,0.0,1.0,1,1,0,0,1,4.0,18,15,0
4,9,809208,1.0,1.0,1.0,1,1,0,0,1,1.0,5,18,0


In [9]:
# lda jobs features
features_job = ['ReqTopic', 'DescTopic', 'TitTopic']
# lda & user profile features
features_user = ['DegreeType', 'CurrentlyEmployed', 'ManagedOthers', 'WorkHistoryTopic',
       'WorkHistoryLevel', 'SeniorLevel']
# location features
features_location = ['City', 'State', 'Country']

In [10]:
from topN_ebm_random import *

In [ ]:
# !cat topN_ebm_random.py

In [13]:
user_fm = pd.read_csv('./data_interim/users_fm.csv')
user_fm.head()

,UserID,Split,DegreeType,CurrentlyEmployed,ManagedOthers,WorkHistoryTopic,WorkHistoryLevel,SeniorLevel
0,80,Train,1,1,1,9,2,4.0
1,123,Train,4,1,0,0,0,3.0
2,162,Train,5,0,0,0,3,6.0
3,178,Train,1,0,1,0,3,6.0
4,344,Train,1,1,0,0,1,3.0


In [14]:
job_fm = pd.read_csv('./data_interim/jobs_fm.csv')
job_fm.head()

,JobID,ReqTopic,DescTopic,TitTopic
0,1,5,15,0
1,4,18,15,0
2,7,15,18,0
3,8,15,7,0
4,9,15,7,0


# Start topN for a given user_id

- features_job_matrix: feature names for job matrix
- features_work_matrix: feature names for work history matrix
- user_profile_features: list of features for user profile

In [15]:
%%time
import gzip, pickle, pickletools

filepath = "./output_ebm/ebm_extended_dropid.pikle"

with gzip.open(filepath, 'rb') as f:
    p = pickle.Unpickler(f)
    ebm_extended = p.load()

CPU times: user 1.61 ms, sys: 1.31 ms, total: 2.92 ms
Wall time: 2.69 ms


In [16]:
%%time
rec_result_ebm_extended = topN_ebm_extended(model = ebm_extended, 
           N = 20, 
           ranking_data = ranking_data, 
           user_fm=user_fm, job_fm=job_fm)

CPU times: user 57min 42s, sys: 1min 22s, total: 59min 4s
Wall time: 50min 14s


In [17]:
# Function: get rec_result corresponding to a user id
# rec_result: overall rec_recsult dictionary (output of topN_flat)
def get_rec_result_df(u_id, rec_result):
    rec_20_df = pd.DataFrame(rec_result[u_id])
    rec_cols = ['JobID', 'Y_prob', 'Y_pred']
    rec_20_df.columns = rec_cols
    rec_20_df['UserID'] = u_id
    rec_20_df['rank'] = rec_20_df.groupby('UserID').cumcount()
    return rec_20_df


In [18]:
user_ids = list(ranking_data.groupby('UserID').UserID.unique().astype('int'))

In [19]:
%%time
final_rec_result = pd.DataFrame(columns = ['JobID', 'Y_prob', 'Y_pred','UserID','rank'])

for u_id in user_ids:
    
    temp_df = get_rec_result_df(u_id=u_id, 
                                rec_result=rec_result_ebm_extended)
    # print(u_id, len(temp_df))
    final_rec_result = pd.concat([final_rec_result, temp_df])

final_rec_result = final_rec_result[['UserID','JobID', 'Y_prob', 'Y_pred', 'rank']]

CPU times: user 14 s, sys: 294 ms, total: 14.3 s
Wall time: 14.9 s


In [20]:
final_rec_result.head()

,UserID,JobID,Y_prob,Y_pred,rank
0,13,821691,0.937830,1,0
1,13,612698,0.478777,0,1
2,13,472398,0.454115,0,2
3,13,234659,0.454115,0,3
4,13,145443,0.454115,0,4


In [21]:
final_rec_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74320 entries, 0 to 19
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   UserID  74320 non-null  object 
 1   JobID   74320 non-null  object 
 2   Y_prob  74320 non-null  float64
 3   Y_pred  74320 non-null  object 
 4   rank    74320 non-null  object 
dtypes: float64(1), object(4)
memory usage: 3.4+ MB


In [22]:
final_rec_result.to_csv('./output_topN_ebm/rec_result_ebm_extended_random.csv', header=True, index=False)